In [1]:
import numpy as np
import sqlite3
import wikidata_utils as wdutils
import pandas as pd
import ast
from sample_size import sample_size

import qwikidata
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

from tqdm.auto import tqdm
tqdm.pandas()

SEED=42

from IPython.display import clear_output

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/gabriela/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
#from importlib import reload
#wdAPI.save_entity_cache(force=True)
#reload(wdutils)

In [3]:
wdAPI = wdutils.CachedWikidataAPI(
    cache_path = '../wikidata_entity_cache.p',
    save_every_x_queries=np.inf
)

In [4]:
db = sqlite3.connect('../wikidata_claims_refs_parsed.db')
cursor = db.cursor()
claims_columns = ['entity_id','claim_id','rank','property_id','datatype','datavalue']
refs_columns = ['reference_id', 'reference_property_id', 'reference_index', 'reference_datatype', 'reference_value']

In [5]:
# Checking first few elements
cursor.execute('select * from refs limit 5;')
head_df = pd.DataFrame(cursor.fetchall())
head_df.columns = refs_columns
head_df

,reference_id,reference_property_id,reference_index,reference_datatype,reference_value
0,7c6b234780b3ee79e341952bbe69fd0e93298176,P854,0,url,{'value': 'http://gwpapers.virginia.edu/histor...
1,7c6b234780b3ee79e341952bbe69fd0e93298176,P123,0,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
2,dd6c0c287c2d4dfd674f577b65ac1d6b875005ae,P248,0,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
3,dd6c0c287c2d4dfd674f577b65ac1d6b875005ae,P304,0,string,"{'value': '389', 'type': 'string'}"
4,21e05588017fe6569fd5fb71be9fc6b97812a49b,P248,0,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."


In [6]:
text_reference_sampled_df_html = pd.read_csv('text_reference_sampled_df_html.csv')
text_reference_sampled_df_html

,reference_id,reference_property_id,reference_datatype,url,netloc,netloc_agg,error_msg,code,content-type,final_url,reason,language_crawl,language_crawl_score,sampling_weight_vb,sampling_weight,html
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,P4656,url,https://en.wikipedia.org/w/index.php?title=G._...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=G._...,OK,en,0.934613,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,P4656,url,https://en.wikipedia.org/w/index.php?title=Seb...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Seb...,OK,en,0.646651,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,P4656,url,https://en.wikipedia.org/w/index.php?title=Ave...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Ave...,OK,en,0.842498,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
3,76b04346ad57869d9e5ae1007ba8343d708ab6f9,P4656,url,https://en.wikipedia.org/w/index.php?title=Yut...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Yut...,OK,en,0.608352,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
4,1ee71b39caf6df395c64b436fea4895692812d38,P4656,url,https://en.wikipedia.org/w/index.php?title=Dol...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Dol...,OK,en,0.856581,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,e004706cba4821386dc31a86faeb680e8fff4ae7,P854,url,https://indiancine.ma/ATB,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/ATB,OK,en,0.540405,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
672,c714ca621d1fe1ca35f20e01a72587d23bdd8e1e,P854,url,https://indiancine.ma/BTW,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/BTW,OK,en,0.712387,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
673,52968f905e88f2484e3cb43e6ab4e76f120bc3c3,P854,url,https://indiancine.ma/BTD,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/BTD,OK,en,0.600702,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
674,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,P854,url,https://indiancine.ma/BKF,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/BKF,OK,en,0.921826,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."


In [7]:
def reference_id_to_claim_id(reference_id):
    try:
        np.random.seed(SEED)
        cursor.execute(f'select claim_id from claims_refs where reference_id="{reference_id}"')
        sql_result = cursor.fetchall()
        #return sql_result
        randomly_chosen_claim_id = np.array(sql_result).reshape(-1)
        return randomly_chosen_claim_id
    except Exception:
        print(reference_id)
        print(sql_result)
        raise
        
def reference_id_to_claim_data(reference_id):
    claim_ids = reference_id_to_claim_id(reference_id)
    r = []
    for claim_id in claim_ids:
        #print(claim_id)
        cursor.execute(f'select * from claims where claim_id="{claim_id}";')
        d = cursor.fetchall()
        r = r + d
    return r

claim_data = []
for reference_id in text_reference_sampled_df_html.reference_id:
    data = reference_id_to_claim_data(reference_id)    
    #print(data)
    data = [(reference_id,) + t for t in data]
    claim_data = claim_data + data
    #break

In [8]:
claim_df = pd.DataFrame(claim_data, columns = ['reference_id'] + claims_columns)
claim_df

,reference_id,entity_id,claim_id,rank,property_id,datatype,datavalue
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,Q5512528,Q5512528$81E8AD02-28AF-4AE3-8ACD-047C30B40B01,normal,P20,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,Q12149940,Q12149940$C9FE5F0C-78FD-4ECE-B1C2-16A763B8ED4E,normal,P140,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,Q583556,Q583556$929DCD2F-3B4A-41B5-89E7-EEC549077834,normal,P740,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
3,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,Q583556,Q583556$2AD75C92-953E-47CB-93A6-D082674F4231,normal,P2031,time,"{'value': {'time': '+1972-00-00T00:00:00Z', 't..."
4,76b04346ad57869d9e5ae1007ba8343d708ab6f9,Q7586053,Q7586053$AD344012-A522-4A8A-AF64-16F859E2EA9A,normal,P2031,time,"{'value': {'time': '+1983-00-00T00:00:00Z', 't..."
...,...,...,...,...,...,...,...
1761,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,Q58484448,Q58484448$9611AF89-1BD2-4D49-9CBB-E3EC52C7765D,normal,P364,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
1762,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,Q58484448,Q58484448$E03C8ED8-0593-4D47-B002-6371900A238E,normal,P462,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
1763,6e6204011de4f4176362a6f4067486a94fc6ac3e,Q58435606,Q58435606$474B340F-3E0A-451C-8844-C02092719DD0,normal,P31,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'..."
1764,6e6204011de4f4176362a6f4067486a94fc6ac3e,Q58435606,Q58435606$CF542729-F114-4073-BB05-98D2EDEE5AD0,normal,P5987,external-id,"{'value': 'YI', 'type': 'string'}"


In [9]:
#reference_id_to_claim_data('93c70463e9b27bd9a1d62a170b23cc55ced0f7a4')

#cursor.execute('select * from claims_refs where reference_id=\'ca5a493f7843383b127c7cff4f7c9c26af9b2f87\'')
#sql_result = cursor.fetchall()
#sql_result

In [10]:
def claim_id_to_claim_url(claim_id):
    claim_id_parts = claim_id.split('$')
    return f'https://www.wikidata.org/wiki/{claim_id_parts[0]}#{claim_id}'

In [11]:
#print(claim_id_to_claim_url('Q58481743$72999E39-348D-4DD3-8272-D4A24B9E7894'))

In [12]:
claim_df.datatype.value_counts()

wikibase-item       926
time                389
external-id         149
string              101
quantity             91
monolingualtext      82
url                  15
globe-coordinate     12
commonsMedia          1
Name: datatype, dtype: int64

In [13]:
BAD_DATATYPES = ['external-id','commonsMedia','url', 'globe-coordinate', 'wikibase-lexeme', 'wikibase-property']

assert claim_df[~claim_df.datatype.isin(BAD_DATATYPES)].reference_id.unique().shape\
    == claim_df.reference_id.unique().shape

print(claim_df.reference_id.unique().shape[0])

676


In [14]:
claim_df = claim_df[~claim_df.datatype.isin(BAD_DATATYPES)].reset_index(drop=True)

In [15]:
from tqdm.auto import tqdm
tqdm.pandas()

claim_df[['entity_label','entity_label_lan']] = pd.DataFrame(
    claim_df.entity_id.progress_apply(wdAPI.get_label, non_language_set=True).tolist()
)
claim_df[['property_label','property_label_lan']] = pd.DataFrame(
    claim_df.property_id.progress_apply(wdAPI.get_label, non_language_set=True).tolist()
)

claim_df[['entity_alias','entity_alias_lan']] = pd.DataFrame(
    claim_df.entity_id.progress_apply(wdAPI.get_alias, non_language_set=True).tolist()
)
claim_df[['property_alias','property_alias_lan']] = pd.DataFrame(
    claim_df.property_id.progress_apply(wdAPI.get_alias, non_language_set=True).tolist()
)

claim_df[['entity_desc','entity_desc_lan']] = pd.DataFrame(
    claim_df.entity_id.progress_apply(wdAPI.get_desc, non_language_set=True).tolist()
)
claim_df[['property_desc','property_desc_lan']] = pd.DataFrame(
    claim_df.property_id.progress_apply(wdAPI.get_desc, non_language_set=True).tolist()
)

/home/gabriela/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
claim_df[['object_label','object_label_lan']] = pd.DataFrame(
    claim_df.progress_apply(lambda x : wdAPI.get_object_label_given_datatype(x['datatype'], x['datavalue']), axis=1).tolist()
)

claim_df[['object_alias','object_alias_lan']] = pd.DataFrame(
    claim_df.progress_apply(lambda x : wdAPI.get_object_alias_given_datatype(x['datatype'], x['datavalue']), axis=1).tolist()
)

claim_df[['object_desc','object_desc_lan']] = pd.DataFrame(
    claim_df.progress_apply(lambda x : wdAPI.get_object_desc_given_datatype(x['datatype'], x['datavalue']), axis=1).tolist()
)

In [17]:
claim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reference_id        1589 non-null   object
 1   entity_id           1589 non-null   object
 2   claim_id            1589 non-null   object
 3   rank                1589 non-null   object
 4   property_id         1589 non-null   object
 5   datatype            1589 non-null   object
 6   datavalue           1589 non-null   object
 7   entity_label        1589 non-null   object
 8   entity_label_lan    1589 non-null   object
 9   property_label      1589 non-null   object
 10  property_label_lan  1589 non-null   object
 11  entity_alias        1589 non-null   object
 12  entity_alias_lan    1589 non-null   object
 13  property_alias      1589 non-null   object
 14  property_alias_lan  1589 non-null   object
 15  entity_desc         1589 non-null   object
 16  entity_desc_lan     1589

In [18]:
# Removing bad object labels
claim_df = claim_df[claim_df['object_label_lan'] != 'no_lan'].reset_index(drop=True)
print(claim_df.reference_id.unique().shape[0])

676


In [19]:
for row in claim_df.apply(lambda x : x['property_id']+'_'+x['property_label'], axis=1)\
    .value_counts().reset_index().iterrows():
    print(row)

(0, index    P31_instance of
0                    158
Name: 0, dtype: object)
(1, index    P569_date of birth
0                       155
Name: 1, dtype: object)
(2, index    P570_date of death
0                       118
Name: 2, dtype: object)
(3, index    P39_position held
0                       65
Name: 3, dtype: object)
(4, index    P1476_title
0                 44
Name: 4, dtype: object)
(5, index    P571_inception
0                    43
Name: 5, dtype: object)
(6, index    P186_made from material
0                             43
Name: 6, dtype: object)
(7, index    P217_inventory number
0                           42
Name: 7, dtype: object)
(8, index    P195_collection
0                     40
Name: 8, dtype: object)
(9, index    P136_genre
0                37
Name: 9, dtype: object)
(10, index    P21_sex or gender
0                       37
Name: 10, dtype: object)
(11, index    P276_location
0                   34
Name: 11, dtype: object)
(12, index    P17_country
0         

Name: 121, dtype: object)
(122, index    P527_has part
0                    1
Name: 122, dtype: object)
(123, index    P2993_partition coefficient water/octanol
0                                                1
Name: 123, dtype: object)
(124, index    P175_performer
0                     1
Name: 124, dtype: object)
(125, index    P162_producer
0                    1
Name: 125, dtype: object)
(126, index    P3919_contributed to creative work
0                                         1
Name: 126, dtype: object)
(127, index    P828_has cause
0                     1
Name: 127, dtype: object)
(128, index    P281_postal code
0                       1
Name: 128, dtype: object)
(129, index    P2177_solubility
0                       1
Name: 129, dtype: object)
(130, index    P2102_boiling point
0                          1
Name: 130, dtype: object)
(131, index    P1889_different from
0                           1
Name: 131, dtype: object)
(132, index    P451_unmarried partner
0               

In [20]:
claim_df.to_csv('text_reference_claims_df.csv', index=None)

In [2]:
claim_df = pd.read_csv('text_reference_claims_df.csv')

In [3]:
claim_df

,reference_id,entity_id,claim_id,rank,property_id,datatype,datavalue,entity_label,entity_label_lan,property_label,...,entity_desc,entity_desc_lan,property_desc,property_desc_lan,object_label,object_label_lan,object_alias,object_alias_lan,object_desc,object_desc_lan
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,Q5512528,Q5512528$81E8AD02-28AF-4AE3-8ACD-047C30B40B01,normal,P20,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",G. V. Raja,en,place of death,...,sports official,en,most specific known (e.g. city instead of coun...,en,Kullu Valley,en,"['Kulû', 'Vallee de Kulu', 'Vallée de kulu']",fr,"valley in Himachal Pradesh, India",en
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,Q12149940,Q12149940$C9FE5F0C-78FD-4ECE-B1C2-16A763B8ED4E,normal,P140,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Sebastian Sabol,en,religion,...,"Ukrainian Basilian priest, poet and writer (19...",en,"religion of a person, organization or religiou...",en,Greek catholic church,en,"['Iglesia Católica Griega', 'Iglesia catolica ...",es,group of Eastern Catholic Churches following t...,en
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,Q583556,Q583556$929DCD2F-3B4A-41B5-89E7-EEC549077834,normal,P740,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Average White Band,en,location of formation,...,Scottish R&B band,en,location where a group or organization was formed,en,Dundee,en,['City of Dundee'],en,city in Scotland,en
3,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,Q583556,Q583556$2AD75C92-953E-47CB-93A6-D082674F4231,normal,P2031,time,"{'value': {'time': '+1972-00-00T00:00:00Z', 't...",Average White Band,en,work period (start),...,Scottish R&B band,en,start of period during which a person or group...,en,1972,en,no-alias,none,no-desc,none
4,76b04346ad57869d9e5ae1007ba8343d708ab6f9,Q7586053,Q7586053$AD344012-A522-4A8A-AF64-16F859E2EA9A,normal,P2031,time,"{'value': {'time': '+1983-00-00T00:00:00Z', 't...",Yutaka Higuchi,en,work period (start),...,Japanese musician,en,start of period during which a person or group...,en,1983,en,no-alias,none,no-desc,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,Q58484448,Q58484448$DB944078-CA87-4617-B942-5B290F2144ED,normal,P31,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Gul Sanobar,en,instance of,...,1934 film,en,that class of which this subject is a particul...,en,film,en,"['movie', 'motion picture', 'cinematic work', ...",en,audiovisual work,en
1568,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,Q58484448,Q58484448$9611AF89-1BD2-4D49-9CBB-E3EC52C7765D,normal,P364,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Gul Sanobar,en,original language of film or TV show,...,1934 film,en,language in which a film or a performance work...,en,Hindi,en,"['Hindi language', 'Modern Standard Hindi', 'hi']",en,Indo-Aryan language,en
1569,8bb93e520f7ca8208f24078434b8f5f2ba8b520b,Q58484448,Q58484448$E03C8ED8-0593-4D47-B002-6371900A238E,normal,P462,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Gul Sanobar,en,color,...,1934 film,en,color of subject,en,black and white,en,"['B/W', 'B&W', 'colorless cinema', 'black-and-...",en,monochrome form in visual arts,en
1570,6e6204011de4f4176362a6f4067486a94fc6ac3e,Q58435606,Q58435606$474B340F-3E0A-451C-8844-C02092719DD0,normal,P31,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...",Jai Bhawani,en,instance of,...,1928 film,en,that class of which this subject is a particul...,en,film,en,"['movie', 'motion picture', 'cinematic work', ...",en,audiovisual work,en


In [23]:
wdAPI.save_entity_cache(force=True)

In [7]:
claim_df[claim_df['rank'] == 'deprecated']

,reference_id,entity_id,claim_id,rank,property_id,datatype,datavalue,entity_label,entity_label_lan,property_label,...,entity_desc,entity_desc_lan,property_desc,property_desc_lan,object_label,object_label_lan,object_alias,object_alias_lan,object_desc,object_desc_lan
153,5bc0f775565a29c227394e2f1888dda61f8efa1e,Q6254987,Q6254987$05e8ce3a-2d97-4ffa-bdae-b18e0789ac2a,deprecated,P570,time,"{'value': {'time': '+1836-12-12T00:00:00Z', 't...",John Rippon,en,date of death,...,British minister,en,date on which the subject died,en,12/12/1836,en,"['12 of December, 1836', '12/12/1836 (dd/mm/yy...",en,no-desc,none
1448,5c8727f48dfdeef75592102281f6a0efd6da86d9,Q28048529,Q28048529$9e00721e-4532-344a-5e34-2893e3ae13e8,deprecated,P6216,wikibase-item,"{'value': {'entity-type': 'item', 'numeric-id'...","William Wyndham Grenville, 1st Baron Grenville",en,copyright status,...,painting by John Hoppner,en,copyright status for intellectual creations li...,en,copyrighted,en,"['in copyright', 'under copyright']",en,legal state of a work as recognised as an inte...,en
